In [41]:
import pandas as pd
import json
import ast
import nltk

### file to analyze = steam_games 

In [42]:
# list 'output_steam_games.json'
rows = []
with open ('data/steam_games.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(json.loads(line))

# save to df
df_steam_games = pd.DataFrame(rows)

In [43]:
df_steam_games.dropna(how='all', inplace=True)
df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [44]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 3.4+ MB


In [45]:
df_steam_games.isna().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [46]:
# fill na values of 'title' with 'app_name' values
df_steam_games['title'].fillna(df_steam_games['app_name'], inplace=True)

### genres

In [47]:
df_steam_games.isna().sum()

publisher       8052
genres          3283
app_name           2
title              2
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [48]:
df_steam_games['genres'].fillna(df_steam_games['tags'], inplace=True)


In [49]:
# df_steam_games['genres'].unique()

In [54]:
# Fill NaN values with an empty list
df_steam_games['genres'] = df_steam_games['genres'].fillna('')
df_steam_games['tags'] = df_steam_games['tags'].fillna('')

# Define a function to handle the conversion
def convert_to_string(x):
    if isinstance(x, list):
        return x
    else:
        return x.split(', ')
    
# Apply this function to the 'genres' and 'tags' columns
df_steam_games['genres'] = df_steam_games['genres'].apply(convert_to_string)
df_steam_games['tags'] = df_steam_games['tags'].apply(convert_to_string)

# Concatenate the 'genres' and 'tags' columns
df_steam_games['genres'] = df_steam_games['genres'] + df_steam_games['tags']

# Convert to a set to remove duplicates, then convert back to a list
df_steam_games['genres'] = df_steam_games['genres'].apply(lambda x: list(set(x)))

In [60]:
# Initialize an empty set to store the unique genres
unique_genres = set()

# Iterate over the 'genres' column
for genres_list in df_steam_games['genres']:
    # Add the genres from this row to the set
    unique_genres.update(genres_list)

# Now 'unique_genres' contains all unique genres
unique_genres

{'',
 '1980s',
 "1990's",
 '2.5D',
 '2D',
 '2D Fighter',
 '360 Video',
 '3D Platformer',
 '3D Vision',
 '4 Player Local',
 '4X',
 '6DOF',
 'Abstract',
 'Accounting',
 'Action',
 'Action RPG',
 'Action-Adventure',
 'Adventure',
 'Agriculture',
 'Aliens',
 'Alternate History',
 'America',
 'Animation & Modeling',
 'Animation &amp; Modeling',
 'Anime',
 'Arcade',
 'Arena Shooter',
 'Artificial Intelligence',
 'Assassin',
 'Asynchronous Multiplayer',
 'Atmospheric',
 'Audio Production',
 'Base Building',
 'Based On A Novel',
 'Basketball',
 'Batman',
 "Beat 'em up",
 'Benchmark',
 'Bikes',
 'Blood',
 'Board Game',
 'Bowling',
 'Building',
 'Bullet Hell',
 'Bullet Time',
 'CRPG',
 'Capitalism',
 'Card Game',
 'Cartoon',
 'Cartoony',
 'Casual',
 'Character Action Game',
 'Character Customization',
 'Chess',
 'Choices Matter',
 'Choose Your Own Adventure',
 'Cinematic',
 'City Builder',
 'Class-Based',
 'Classic',
 'Clicker',
 'Co-op',
 'Co-op Campaign',
 'Cold War',
 'Colorful',
 'Comedy',
 

In [56]:
df_steam_games['genres'].unique()

TypeError: unhashable type: 'list'

In [34]:
df_steam_games['genres'].unique()

array(["'Action', 'Simulation'], 'Indie', 'Simulation', ['Strategy', ['Action', 'Strategy'], 'Casual'",
       "'RPG', '2D', 'Indie', ['Free to Play', 'PvP', 'Design & Illustration'], 'Replay Value', 'Competitive', 'Tactical', 'Fantasy', 'Character Customization', 'Card Game', 'Board Game', 'Dark Fantasy', 'Difficult', 'Turn-Based', 'Trading Card Game', 'Strategy', 'Strategy'], 'Female Protagonist'",
       "['Casual', 'Multiplayer'], 'Indie', 'Simulation', ['Free to Play', 'Sports'], 'Casual', 'Sports', 'Free to Play'",
       ..., "'RPG', 'Puzzle', 'Tower Defense'], 'RPG'], ['Indie'",
       "'Violent', 'Indie'], 'Adventure'], 'Indie', ['Action', 'Casual', 'Adventure'",
       "'Puzzle', 'Indie'], ['Casual', 'Singleplayer', 'Atmospheric', 'Relaxing'], ['Indie', 'Casual'"],
      dtype=object)

In [58]:
# df_steam_games['genres'].unique()
# Initialize an empty set to store the unique genres
unique_genres = set()

# Iterate over the 'genres' column
for genres in df_steam_games['genres']:
    # Split the string into individual genres and add them to the set
    unique_genres.update(genres.split(', '))

# Now 'unique_genres' contains all unique genres
print(unique_genres)

AttributeError: 'list' object has no attribute 'split'

In [38]:
# Primero comprobe que la columna genres y tags, contienen los mismos valores, complete los faltantes de la columna tags con los valores de la columna genres
filas_vacias = df_steam_games['tags'].isna() 
df_steam_games.loc[filas_vacias, 'tags'] = df_steam_games.loc[filas_vacias, 'genres'] 
df_steam_games.drop(columns='genres',inplace=True) 

def extract(x):
    # Use list comprehension to split each string and strip unwanted characters
    filtro = [y.strip(" '[]'") for x in x for y in x.split(',')]
    return list(set(filtro))

# Apply the function to the 'tags' column
df_steam_games['tags'] = df_steam_games['tags'].apply(extract)

In [40]:
df_steam_games['tags'].unique()

TypeError: unhashable type: 'list'

In [20]:
extract(df_steam_games['tags'])

['Soccer',
 'Lara Croft',
 'Heist',
 'World War I',
 'Interactive Fiction',
 'Fighting',
 'Board Game',
 'Magic',
 'Music-Based Procedural Generation',
 'Economy',
 'Dragons',
 'Bikes',
 'FMV',
 'FPS',
 'Animation & Modeling',
 'Violent',
 'Tanks',
 'Building',
 'Turn-Based Strategy',
 'Exploration',
 'nan',
 'Wrestling',
 'Singleplayer',
 'Third Person',
 'Crime',
 'Mystery',
 'Simulation',
 'Top-Down',
 'Software Training',
 'Multiplayer',
 'Western',
 'Blood',
 'Lovecraftian',
 'Replay Value',
 'Cartoony',
 'Capitalism',
 'Female Protagonist',
 'Real-Time',
 'Early Access',
 'First-Person',
 'Parody',
 'Dark Comedy',
 'Hand-drawn',
 'Puzzle',
 'Crowdfunded',
 'Perma Death',
 'Cute',
 'Pool',
 'Detective',
 'Memes',
 'Short',
 'Otome',
 'Touch-Friendly',
 'Tactical RPG',
 'Warhammer 40K',
 'Multiple Endings',
 'Tactical',
 'Golf',
 'Photo Editing',
 'Conspiracy',
 'Asynchronous Multiplayer',
 'God Game',
 'Runner',
 'Diplomacy',
 'Spelling',
 'Zombies',
 'Medieval',
 'Offroad',
 '"Sh

In [47]:
df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"'Action', 'Indie', ['Strategy', ['Action', 'Ca...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","'Fantasy', 'Trading Card Game', 'Tactical', 'T...",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"'Indie', 'Multiplayer'], ['Casual', 'Sports'],...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"'Casual'], 'Adventure', ['Action'",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,"'Indie', 'Sports'], 'Casual', ['Action'",Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"'Indie', ['Casual', ['Strategy', 'Casual', 'Si...",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"'Indie', ['Casual', ['Strategy', 'Casual'], 'S...",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation', 'Racing'], '...",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"'Indie'], 'Puzzle', ['Indie', ['Casual', 'Casu...",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [48]:
# find missing id and app_name
missing = df_steam_games[df_steam_games[['id', 'app_name']].isna().any(axis=1)]
missing

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,NaN,nan,NaN,NaN,http://store.steampowered.com/,NaN,nan,NaN,NaN,19.99,False,NaN,NaN
90890,NaN,"'Indie'], ['Action'",NaN,NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"['Action', 'Indie']",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",NaN,False,317160,NaN
119271,"Warner Bros. Interactive Entertainment, Feral ...","'Metroidvania', 'Sandbox'], 'Atmospheric', 'Ad...",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"['Action', 'Open World', 'Batman', 'Adventure'...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [49]:
# Extract the id of the URL for Batman
df_steam_games.loc[
    df_steam_games['title'] == 'Batman: Arkham City - Game of the Year Edition', 'id'
    ] = df_steam_games['url'].apply(lambda x: x[-6:])


In [50]:
# Delete 
df_steam_games = df_steam_games[df_steam_games['url'] != 'http://store.steampowered.com/']
df_steam_games = df_steam_games[df_steam_games['id'] != '317160']

In [51]:
# checking missing again
missing = df_steam_games[df_steam_games[['id', 'app_name']].isna().any(axis=1)]
missing

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


In [52]:
# Check content for some columns, select genres to work with
print(df_steam_games['genres'])

88310     'Action', 'Indie', ['Strategy', ['Action', 'Ca...
88311     'Fantasy', 'Trading Card Game', 'Tactical', 'T...
88312     'Indie', 'Multiplayer'], ['Casual', 'Sports'],...
88313                     'Casual'], 'Adventure', ['Action'
88314               'Indie', 'Sports'], 'Casual', ['Action'
                                ...                        
120440    'Indie', ['Casual', ['Strategy', 'Casual', 'Si...
120441    'Indie', ['Casual', ['Strategy', 'Casual'], 'S...
120442    ['Indie', 'Racing', 'Simulation', 'Racing'], '...
120443    'Indie'], 'Puzzle', ['Indie', ['Casual', 'Casu...
120444    'VR'], 'Indie', 'Action', 'Adventure', 'Simula...
Name: genres, Length: 32133, dtype: object


### price

In [53]:
df_steam_games['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, nan, 'Free', 10.99, 1.59, 14.99, 1.99, 59.99, 8.99,
       6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25,
       24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99,
       179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99,
       22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95,
       499.99, 16.06, 4.68, 131

In [54]:
df_steam_games.dropna(subset='price', inplace=True)

In [55]:
df_steam_games['price'].isnull().sum()

0

In [56]:
df_steam_games['price'] = df_steam_games['price'].replace(r'.*Free.*', '0.00', regex=True)
df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"'Action', 'Indie', ['Strategy', ['Action', 'Ca...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","'Fantasy', 'Trading Card Game', 'Tactical', 'T...",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL
88312,Poolians.com,"'Indie', 'Multiplayer'], ['Casual', 'Sports'],...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Poolians.com
88313,彼岸领域,"'Casual'], 'Adventure', ['Action'",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,"'Indie', 'Sports'], 'Casual', ['Action'",Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"'Indie', ['Casual', ['Strategy', 'Casual', 'Si...",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"'Indie', ['Casual', ['Strategy', 'Casual'], 'S...",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation', 'Racing'], '...",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"'Indie'], 'Puzzle', ['Indie', ['Casual', 'Casu...",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [57]:
df_steam_games['price'].unique()

array([4.99, '0.00', 0.99, 2.99, 3.99, 9.99, 18.99, 29.99, 10.99, 1.59,
       14.99, 1.99, 59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99,
       5.99, 2.49, 15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99,
       34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99,
       49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99, 149.99, 771.71,
       'Install Now', 21.99, 89.99, 'Play WARMACHINE: Tactics Demo', 0.98,
       139.92, 4.29, 64.99, 54.99, 74.99, 'Install Theme', 0.89,
       'Third-party', 0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49,
       23.99, 49.0, 20.99, 10.93, 1.39, 36.99, 4.49, 2.0, 4.0, 9.0,
       234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99, 179.0,
       26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99, 22.99,
       320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95, 499.99,
       16.06, 4.68, 131.4, 44.98, 202.76, 1.0, 2.3, 0.95, 172.24, 249.99,
       2.97, 10.96, 10.0, 30.0, 2.66, 6.48, 19.29, 11.15, 18.9, 2.89,
  

In [58]:
# Convert all non-float values to '0.00'
df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')
df_steam_games['price'].fillna('0.00', inplace=True)

# Replace 'Starting at $...' values with the price after it
# df_steam_games['price'] = df_steam_games['price'].replace(r'^Starting at \$([0-9\.]+).*$', r'\1', regex=True)
# df_steam_games.loc[df_steam_games['price'].str.startswith('Starting at $'), 'price'] = df_steam_games['price'].str.extract('Starting at \$([0-9\.]+)', expand=False)
# df_steam_games['price'] = df_steam_games['price'].replace(r'^Starting at $([0-9\.]+).*$', lambda x: x.group(1), regex=True)

df_steam_games['price'] = df_steam_games['price'].replace('Starting at $499.00', 499.00)
df_steam_games['price'] = df_steam_games['price'].replace('Starting at $449.00', 449.00)

In [59]:
df_steam_games['price'].unique()

array([4.99, 0.0, 0.99, 2.99, 3.99, 9.99, 18.99, 29.99, 10.99, 1.59,
       14.99, 1.99, 59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99,
       5.99, 2.49, 15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99,
       34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99,
       49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99, 149.99, 771.71,
       '0.00', 21.99, 89.99, 0.98, 139.92, 4.29, 64.99, 54.99, 74.99,
       0.89, 0.5, 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.39, 36.99, 4.49, 2.0, 4.0, 9.0, 234.99, 1.95, 1.5, 199.0,
       189.0, 6.66, 27.99, 10.49, 129.99, 179.0, 26.99, 399.99, 31.99,
       399.0, 20.0, 40.0, 3.33, 199.99, 22.99, 320.0, 38.85, 71.7, 59.95,
       995.0, 27.49, 3.39, 6.0, 19.95, 499.99, 16.06, 4.68, 131.4, 44.98,
       202.76, 1.0, 2.3, 0.95, 172.24, 249.99, 2.97, 10.96, 10.0, 30.0,
       2.66, 6.48, 19.29, 11.15, 18.9, 2.89, 99.0, 87.94, 599.0, 8.98,
       9.69, 0.49, 9.98, 9.95, 7.0, 12.89, 6.49, 1.87, 42.99, 41.99,
 

# developer

In [60]:
df_steam_games['developer'].fillna(df_steam_games['publisher'], inplace=True)

In [61]:
df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"'Action', 'Indie', ['Strategy', ['Action', 'Ca...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","'Fantasy', 'Trading Card Game', 'Tactical', 'T...",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.0,False,643980,Secret Level SRL
88312,Poolians.com,"'Indie', 'Multiplayer'], ['Casual', 'Sports'],...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.0,False,670290,Poolians.com
88313,彼岸领域,"'Casual'], 'Adventure', ['Action'",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,"'Indie', 'Sports'], 'Casual', ['Action'",Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"'Indie', ['Casual', ['Strategy', 'Casual', 'Si...",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"'Indie', ['Casual', ['Strategy', 'Casual'], 'S...",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation', 'Racing'], '...",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"'Indie'], 'Puzzle', ['Indie', ['Casual', 'Casu...",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [62]:
# drop the columns that dont have importance to the model 
df_steam_games = df_steam_games.drop(['publisher', 'app_name', 'url', 'tags', 'reviews_url', 'specs', 'early_access', 'developer'], axis=1)

In [63]:
df_steam_games.isna().sum()

genres             0
title              0
release_date    1936
price              0
id                 0
dtype: int64

In [64]:
# Encuentra las filas con NaN en 'release_date'
missing_release_date = df_steam_games[df_steam_games['release_date'].isna()]

# Muestra las filas con NaN en 'release_date'
missing_release_date

,genres,title,release_date,price,id
88314,"'Indie', 'Sports'], 'Casual', ['Action'",Log Challenge,NaN,2.99,773570
88321,['Casual'],Icarus Six Sixty Six,NaN,0.0,724910
88329,"'VR'], 'Indie', ['Early Access'",After Life VR,NaN,4.99,772590
88330,"'Action', 'Indie', 'Adventure', 'Casual'], ['E...",Kitty Hawk,NaN,2.99,640250
88332,"'VR'], 'Action', 'Indie', 'Strategy', 'Casual'...",Mortars VR,NaN,0.99,711440
...,...,...,...,...,...
120381,"'Indie', 'Casual'], ['Early Access'",VR Furballs - Demolition,NaN,4.99,704470
120383,"'Military'], 'Strategy', 'VR', ['Action', 'War...",Tank of War-VR,NaN,19.99,745900
120386,"'Funny', ['Casual', 'Comedy', 'VR', 'Difficult...",Flappy Arms,NaN,1.99,764110
120387,"'Casual'], ['Early Access'",SpaceWalker,NaN,0.0,705860


In [65]:
# quisiera poblar los na de genres y release_date con la api de steam

In [66]:
# extract the year of the release_date
df_steam_games['release_date'] = df_steam_games['release_date'].str.extract(r'(\d{4})')

In [67]:
# get dummies of genres 
df_steam_games['genres'] = df_steam_games['genres'].fillna('[]')  # Rellenar los valores faltantes con una lista vacía
df_steam_games['genres'] = df_steam_games['genres'].apply(lambda x: ', '.join(x))  # Convertir la lista de géneros a una cadena separada por comas

# create columns
dummy_genres = df_steam_games['genres'].str.get_dummies(', ')

# Concat
df_steam_games = pd.concat([df_steam_games, dummy_genres], axis=1)

In [68]:
df_steam_games

,genres,title,release_date,price,id,,"""",&,',",",...,p,r,s,t,u,v,w,x,y,z
88310,"', A, c, t, i, o, n, ', ,, , ', I, n, d, i, e...",Lost Summoner Kitty,2018,4.99,761140,1,0,0,1,1,...,0,1,1,1,1,0,0,0,1,0
88311,"', F, a, n, t, a, s, y, ', ,, , ', T, r, a, d...",Ironbound,2018,0.0,643980,1,0,1,1,1,...,1,1,1,1,1,1,0,0,1,1
88312,"', I, n, d, i, e, ', ,, , ', M, u, l, t, i, p...",Real Pool 3D - Poolians,2017,0.0,670290,1,0,0,1,1,...,1,1,1,1,1,0,0,0,1,0
88313,"', C, a, s, u, a, l, ', ], ,, , ', A, d, v, e...",弹炸人2222,2017,0.99,767400,1,0,0,1,1,...,0,1,1,1,1,1,0,0,0,0
88314,"', I, n, d, i, e, ', ,, , ', S, p, o, r, t, s...",Log Challenge,NaN,2.99,773570,1,0,0,1,1,...,1,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,"', I, n, d, i, e, ', ,, , [, ', C, a, s, u, a...",Colony On Mars,2018,1.99,773640,1,0,0,1,1,...,0,1,1,1,1,0,0,0,1,0
120441,"', I, n, d, i, e, ', ,, , [, ', C, a, s, u, a...",LOGistICAL: South Africa,2018,4.99,733530,1,0,0,1,1,...,0,1,1,1,1,0,0,0,1,0
120442,"[, ', I, n, d, i, e, ', ,, , ', R, a, c, i, n...",Russian Roads,2018,1.99,610660,1,0,0,1,1,...,0,0,0,1,1,0,0,0,0,0
120443,"', I, n, d, i, e, ', ], ,, , ', P, u, z, z, l...",EXIT 2 - Directions,2017,4.99,658870,1,0,0,1,1,...,1,1,1,1,1,0,0,1,1,1


In [69]:
# borro  las columnas que no  necesito

df_steam_games = df_steam_games.iloc[:, :-1]
# normalizamos el nombre de algunas columnas
df_steam_games.rename(columns={'Animation &amp; Modeling': 'Animation and Modeling'}, inplace=True)
df_steam_games.rename(columns={'Design &amp; Illustration': 'Design and Illustration'}, inplace=True)
df_steam_games.rename(columns={'id': 'item_id'}, inplace=True)

In [70]:
df_steam_games

,genres,title,release_date,price,item_id,,"""",&,',",",...,o,p,r,s,t,u,v,w,x,y
88310,"', A, c, t, i, o, n, ', ,, , ', I, n, d, i, e...",Lost Summoner Kitty,2018,4.99,761140,1,0,0,1,1,...,1,0,1,1,1,1,0,0,0,1
88311,"', F, a, n, t, a, s, y, ', ,, , ', T, r, a, d...",Ironbound,2018,0.0,643980,1,0,1,1,1,...,1,1,1,1,1,1,1,0,0,1
88312,"', I, n, d, i, e, ', ,, , ', M, u, l, t, i, p...",Real Pool 3D - Poolians,2017,0.0,670290,1,0,0,1,1,...,1,1,1,1,1,1,0,0,0,1
88313,"', C, a, s, u, a, l, ', ], ,, , ', A, d, v, e...",弹炸人2222,2017,0.99,767400,1,0,0,1,1,...,1,0,1,1,1,1,1,0,0,0
88314,"', I, n, d, i, e, ', ,, , ', S, p, o, r, t, s...",Log Challenge,NaN,2.99,773570,1,0,0,1,1,...,1,1,1,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,"', I, n, d, i, e, ', ,, , [, ', C, a, s, u, a...",Colony On Mars,2018,1.99,773640,1,0,0,1,1,...,1,0,1,1,1,1,0,0,0,1
120441,"', I, n, d, i, e, ', ,, , [, ', C, a, s, u, a...",LOGistICAL: South Africa,2018,4.99,733530,1,0,0,1,1,...,0,0,1,1,1,1,0,0,0,1
120442,"[, ', I, n, d, i, e, ', ,, , ', R, a, c, i, n...",Russian Roads,2018,1.99,610660,1,0,0,1,1,...,1,0,0,0,1,1,0,0,0,0
120443,"', I, n, d, i, e, ', ], ,, , ', P, u, z, z, l...",EXIT 2 - Directions,2017,4.99,658870,1,0,0,1,1,...,1,1,1,1,1,1,0,0,1,1


In [71]:
df_steam_games.drop('genres' , axis = 1, inplace = True)

In [72]:
df_steam_games.loc[df_steam_games['item_id'] == '']

,title,release_date,price,item_id,,"""",&,',",",-,...,o,p,r,s,t,u,v,w,x,y


In [73]:
df_steam_games.to_csv('df_steaam_games.csv', index=False)